# This is Constantin research notebook for the project smart city

In [6]:
import requests 
import pandas as pd 

url = 'https://opendata.paris.fr/api/records/1.0/search/?dataset=dans-ma-rue&q=&facet=type&facet=soustype&facet=code_postal&facet=arrondissement&facet=conseilquartier&facet=datedecl&facet=anneedecl&facet=prefixe&facet=intervenant'

dataset = pd.read_csv("dans-ma-rue.csv", sep=";")
dataset.sort_values(by=['DATE DECLARATION'])


,ID DECLARATION,TYPE DECLARATION,SOUS TYPE DECLARATION,ADRESSE,CODE POSTAL,VILLE,ARRONDISSEMENT,CONSEIL DE QUARTIER,DATE DECLARATION,ANNEE DECLARATION,MOIS DECLARATION,OUTIL SOURCE,INTERVENANT,ID_DMR,geo_shape,geo_point_2d
87809,154,Objets abandonnés,Autres objets encombrants abandonnés,"20 Rue du Télégraphe, 75020 PARIS",75020,Paris 20,20,TELEGRAPHE - PELLEPORT - SAINT-FARGEAU,2021-01-01,2021,1,Androï,Ramen en tant que prestataire de DansMaRue,G2021A000154,"{""coordinates"": [2.399587197581498, 48.8726200...","48.87262000128412,2.399587197581498"
664096,225,"Graffitis, tags, affiches et autocollants","Graffitis sur mur, façade sur rue, pont","35 Rue Vieille du Temple, 75004 PARIS",75001,Paris 1,4,SAINT - GERVAIS,2021-01-01,2021,1,IOPage DMR Paris.fr application,graffitis,A2021A000225,"{""coordinates"": [2.357647404947002, 48.8577299...","48.857729995053916,2.357647404947002"
664094,195,Objets abandonnés,Autres objets encombrants abandonnés,"15 Rue Violet, 75015 PARIS",75015,Paris 15,15,DUPLEIX - LA MOTTE PICQUET,2021-01-01,2021,1,Androï,Ramen en tant que prestataire de DansMaRue,G2021A000195,"{""coordinates"": [2.294655304573865, 48.8492099...","48.849209999777266,2.294655304573865"
664093,188,Objets abandonnés,Autres objets encombrants abandonnés,"67 Boulevard Auguste Blanqui, 75013 PARIS",75013,Paris 13,13,BUTTES AUX CAILLES - DAVIEL - BOUSSINGAULT,2021-01-01,2021,1,Androï,Ramen en tant que prestataire de DansMaRue,G2021A000188,"{""coordinates"": [2.348863600928854, 48.8295199...","48.829519999735275,2.348863600928854"
664092,174,Objets abandonnés,Autres objets encombrants abandonnés,"11 bis Rue Curial, 75019 PARIS",75019,Paris 19,19,FLANDRE - AUBERVILLIERS,2021-01-01,2021,1,Androï,Ramen en tant que prestataire de DansMaRue,G2021A000174,"{""coordinates"": [2.371960896362863, 48.8910259...","48.891025999718174,2.371960896362863"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425613,70791,"Graffitis, tags, affiches et autocollants","Graffitis sur mur, façade sur rue, pont","127 Quai de Valmy, 75010 PARIS",75010,Paris 10,10,GRANGE AUX BELLES - TERRAGE,2022-01-30,2022,1,Androï,graffitis,G2022A070791,"{""coordinates"": [2.363983996293543, 48.8762049...","48.876204995067376,2.363983996293543"
338574,70659,"Graffitis, tags, affiches et autocollants","Graffitis sur mur, façade sur rue, pont","43 Rue des Petits Carreaux, 75002 PARIS",75001,Paris 1,2,SENTIER - BONNE NOUVELLE,2022-01-30,2022,1,IOPage DMR Paris.fr application,graffitis,A2022A070659,"{""coordinates"": [2.347420000863303, 48.8680899...","48.86808999888141,2.347420000863303"
604293,70164,"Autos, motos, vélos...",Deux-roues motorisé en stationnement gênant,"12b Rue des Martyrs, 75009 PARIS",75009,Paris 9,9,PIGALLE - MARTYRS,2022-01-30,2022,1,IOPage DMR Paris.fr application,DPMP-SCOP,A2022A070164,"{""coordinates"": [2.33942480008306, 48.87725999...","48.87725999712967,2.33942480008306"
604289,70132,Objets abandonnés,Autres objets encombrants abandonnés,"7 Passage Joanès, 75014 PARIS",75014,Paris 14,14,DIDOT - PORTE DE VANVES,2022-01-30,2022,1,Androï,Ramen en tant que prestataire de DansMaRue,G2022A070132,"{""coordinates"": [2.317360402770434, 48.8283999...","48.82839999630992,2.317360402770434"


In [5]:
dataset.head(1000).to_csv("dans-ma-rue_1000.csv", sep=";")


2021  dataset 
drop id_declaration, geoshape, id-dmr, mois declaration , anne declaration, conseil de quartier, ville, 


2019-2020 
drop objectid, id dmr, ville , anne declaratio , mois dec, numero, prefix, conseil quartier, 
rename: TYPE -> TYPE DECLARATIOn 
        SOUS_TYPE -> SOUS type declaration 
        DATEDECL -> DATE DECLARATION 
        geo_point_2d -> X+Y 

 



In [3]:
from zipfile import ZipFile, Path
import pandas as pd


In [25]:
with ZipFile("DMR_2019.csv.zip") as myzip:
    data1 = myzip.open("DMR_2019.csv")

df1 = pd.read_csv(data1, sep =';',decimal='.' )
df1.columns

Index(['OBJECTID', 'ID_DMR', 'TYPE', 'SOUSTYPE', 'ADRESSE', 'CODE_POSTAL',
       'VILLE', 'ARRONDISSEMENT', 'DATEDECL', 'ANNEEDECL', 'MOISDECL',
       'NUMERO', 'PREFIXE', 'INTERVENANT', 'CONSEILQUARTIER', 'X', 'Y'],
      dtype='object')

In [26]:
with ZipFile("DMR_2020.csv.zip") as myzip:
    data2 = myzip.open("DMR_2020.csv")

df2 = pd.read_csv(data2, sep =';', decimal='.')
df2.columns




Index(['OBJECTID', 'ID_DMR', 'TYPE', 'SOUSTYPE', 'ADRESSE', 'CODE_POSTAL',
       'VILLE', 'ARRONDISSEMENT', 'DATEDECL', 'ANNEEDECL', 'MOISDECL',
       'NUMERO', 'PREFIXE', 'INTERVENANT', 'CONSEILQUARTIER', 'X', 'Y'],
      dtype='object')

In [27]:
df3 = pd.concat([df2, df1])

2021  dataset 
drop id_declaration, geoshape, id-dmr, mois declaration , anne declaration, conseil de quartier, ville, 


2019-2020 
drop objectid, id dmr, ville , anne declaratio , mois dec, numero, prefix, conseil quartier, 
rename: TYPE -> TYPE DECLARATIOn 
        SOUS_TYPE -> SOUS type declaration 
        DATEDECL -> DATE DECLARATION 
        geo_point_2d -> X+Y 

Index(['OBJECTID', 'ID_DMR', 'TYPE', 'SOUSTYPE', 'ADRESSE', 'CODE_POSTAL',
       'VILLE', 'ARRONDISSEMENT', 'DATEDECL', 'ANNEEDECL', 'MOISDECL',
       'NUMERO', 'PREFIXE', 'INTERVENANT', 'CONSEILQUARTIER', 'X', 'Y'],
      dtype='object') 



In [37]:
df3 = df3.drop(['OBJECTID', 'ID_DMR', 'VILLE' ,'ANNEEDECL' ,'MOISDECL' ,'NUMERO','PREFIXE', 'CONSEILQUARTIER',  ], axis = 1)



In [38]:
df3['X'] = df3['X'].astype(str).str.replace(',','.').astype(float)
df3['Y'] = df3['Y'].astype(str).str.replace(',','.').astype(float)


In [40]:
df3.dtypes
    

TYPE               object
SOUSTYPE           object
ADRESSE            object
CODE_POSTAL         int64
ARRONDISSEMENT      int64
DATEDECL           object
INTERVENANT        object
X                 float64
Y                 float64
dtype: object